In [ ]:
import cv2
import numpy as np
import time

# cap = cv2.VideoCapture('vehicle-counting.mp4')
cap = cv2.VideoCapture('C:\Users\Lenovo\Downloads\project colg\project colg\WhatsApp Video 2023-04-17 at 11.10.50.mp4')
# cap = cv2.VideoCapture('vehicle-counting.mp4')

# initialize variables
prev_frame = None
prev_time = None
min_width_rect = 10 # min width rectangle
min_height_rect = 10 # min height rectangle
count_line_position = 325
# Initialize Subtractor
algo = cv2.bgsegm.createBackgroundSubtractorMOG()
car_count = 0
# Load car detection classifier
car_classifier = cv2.CascadeClassifier('cars.xml')

def center_handle(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

detect = []
offset = 4# allowable offset
counter = 0

while True:
    ret, frame1 = cap.read()
    if not ret:
        break
    frame1 = cv2.resize(frame1, (640, 480))
    frame1_copy = frame1.copy()

    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)

    # store the first frame and time
    if prev_frame is None:
        prev_frame = blur
        prev_time = time.time()
        continue

    # calculate the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, blur)
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, hierarchy = cv2.findContours(dilatada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw counting line
    # cv2.line(frame1_copy, (25, count_line_position), (1208, count_line_position), (255, 127, 0), 3)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # validate size of detected object
        validate_counter = (w >= min_width_rect) and (h >= min_height_rect)
        if not validate_counter:
            continue

        # calculate the speed of the car
        car_roi = grey[y-10:y+h+10, x-10:x+w+10]
        car_rects = car_classifier.detectMultiScale(grey, 1.1, 3)

        for (car_x, car_y, car_w, car_h) in car_rects:
            if car_h > 20:
                curr_time = time.time()
                time_diff = curr_time - prev_time
                speed = (car_w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour
                prev_frame = blur
                prev_time = curr_time

            cv2.putText(frame1_copy, f"Speed: {speed:.2f} mph", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Draw rectangle around detected object
            cv2.rectangle(frame1_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # check if the center of the bounding box crossed the counting line
        center_y = int(y + h / 2)
        if center_y >= count_line_position and center_y <= count_line_position + 5:
            car_count += 1

    # Display car count
    cv2.putText(frame1_copy, f"Car count: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Traffic Analysis', frame1_copy)
    if cv2.waitKey(1) == ord('q'):
        break
#Release the capture and close the window
cap.release()
cv2.destroyAllWindows()


: 